In [1]:
import cudf, cugraph

# Load filtered CSVs
nodes_df = cudf.read_csv("papers.csv")  # Contains 'paper_id' and metadata
edges_df = cudf.read_csv("citations.csv")  # Columns: 'src', 'dst'


In [2]:
# Build the graph with renumbering
G = cugraph.Graph(directed=False)
G.from_cudf_edgelist(edges_df, source='src', destination='dst', renumber=True)

# Run Louvain clustering
parts_df, _ = cugraph.louvain(G)

# Un-renumber 'vertex' column
parts_df = G.unrenumber(parts_df, 'vertex')  # now 'vertex' is your original paper_id

# Optionally rename for clarity
parts_df = parts_df.rename(columns={'vertex': 'paper_id'})

# Merge with node metadata
results = parts_df.merge(nodes_df, on='paper_id')

# Export cluster 42
cluster_42 = results[results['partition'] == 42]
cluster_42.to_pandas().to_csv("cluster_42_papers.csv", index=False)


ValueError: Could not convert strings to float type due to presence of non-floating values.

CudaSupportError: Error at driver init: Call to cuInit results in CUDA_ERROR_NO_DEVICE (100)